In [9]:
import pandas as pd 
import numpy as np
pd.read_excel('./../homework/data/momentum_data.xlsx', sheet_name = 'deciles (total returns)').set_index('Date')

,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR
Date,,,,,,,,,,
1927-01-31,-0.0332,-0.0446,0.0269,-0.0033,-0.0041,0.0097,0.0074,0.0036,-0.0041,-0.0024
1927-02-28,0.0753,0.0593,0.0825,0.0727,0.0322,0.0421,0.0283,0.0326,0.0420,0.0701
1927-03-31,-0.0323,-0.0301,-0.0392,-0.0477,-0.0048,-0.0242,0.0205,0.0041,0.0096,0.0547
1927-04-30,0.0204,-0.0320,-0.0247,-0.0137,0.0220,-0.0005,0.0205,-0.0059,0.0159,0.0549
1927-05-31,0.0272,0.0457,0.0595,0.0318,0.0636,0.0580,0.0496,0.0681,0.0811,0.0632
...,...,...,...,...,...,...,...,...,...,...
2024-05-31,0.0562,0.0300,0.0614,0.0105,0.0208,0.0354,0.0451,0.0421,0.0100,0.1182
2024-06-30,-0.0244,0.0045,0.0397,0.0200,0.0030,0.0300,0.0258,0.0358,0.0528,0.0683
2024-07-31,0.1233,0.0358,0.0351,0.0590,0.0444,0.0409,0.0056,0.0152,-0.0092,-0.0469


#### Intepreting the columns: 

- HiPRIOR, LoPRIOR = High Prior Return, Low Prior Return for Momentum Investing strategies
- HiPrior: high returns in the previous period (winners)
- LoPrior: low returns in the previous period (losers)

- SMALL HiPrior: small stock winners
- SMALL LoPrior: small stock losers
- Big HiPrior: big stock winners
- Big LoPrior: big stock losers

#### Understanding the Factors: 
- MKT: Fama French market model
- HML: high - low
- SMB: small - big
- UMD: long and short 

#### Time Series Analysis by Year 


In [10]:
annualization_factor = 12
def portfolio_metrics_basic(returns: pd.DataFrame, annualization_factor: float) -> pd.DataFrame: 
    '''
    Returns Annualized ER, Vol, SR, 
    '''
    
    returns_df = pd.DataFrame(index = returns.columns) 
    mu = returns.mean() * annualization_factor
    vol = returns.std() * np.sqrt(annualization_factor)
    sr = mu / vol

     # skewness -> dimensionless data
    skewness = returns.skew()

    # kurtosis-> dimensionless data
    kurtosis = returns.kurtosis()

    
    returns_df['Annualized Return'] = mu.values 
    returns_df['Annualized Vol'] = vol.values 
    returns_df['Annualized SR'] = sr.values
    returns_df['Skewness'] = skewness

    
    return returns_df

In [ ]:
periods = [['1927', '2024'], ['1927', '1993'], ['1994', '2008'], ['2009', '2024']]
summary_col_names = ['Annualized Return','Annualized Vol','Annualized SR','Skewness']
res = []
for period in periods:
    temp = momentum.loc[period[0]:period[1]]
    temp_ff = factors.loc[period[0]:period[1]]
    summary = portfolio_metrics_basic(temp, annualization_factor)[summary_col_names]
    summary['mkt_corr'] = temp_ff.corr().loc['MKT',['UMD']]
    summary['val_corr'] = temp_ff.corr().loc['HML',['UMD']]
    summary = summary.T.iloc[:,0].rename(f'{period[0]} - {period[1]}')
    res.append(summary)
summary  = pd.concat(res, axis=1).T
summary

##### Deciles: Momentum Trading 

In [ ]:
ls_1decile = deciles['Hi PRIOR'] - deciles['Lo PRIOR']
ls_3decile = deciles.iloc[:,-3:].mean(axis = 1) - deciles.iloc[:,:3].mean(axis = 1)
ls_5decile = deciles.iloc[:,-5:].mean(axis = 1) - deciles.iloc[:,:5].mean(axis = 1)

ls_names = ['mom_D1','mom_D3','mom_D5']
temp = factors.copy()
temp[ls_names] = pd.concat([ls_1decile, ls_3decile, ls_5decile], axis=1)
summary = portfolio_metrics_basic(temp.loc["1994":, ls_names + ['UMD']], annualization_factor)[summary_col_names]
summary['mkt_corr'] = temp.corr().loc['MKT', ls_names + ['UMD']]
summary['val_corr'] = temp.corr().loc['HML', ls_names + ['UMD']]
summary

#### Plot cumprod curves

In [ ]:
(1 + long_only_momentum[mom_names + ['MKT']].loc['1994':]).cumprod().plot()